In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from multiprocess import Pool
from cfs_erf_spatial import erf_spatial
from pytorch_lightning import seed_everything

seed_everything(42);

/Users/sophie/opt/anaconda3/envs/space-env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


### Load Environment

In [2]:
from spacebench import SpaceEnv, DataMaster
datamaster = DataMaster()
datasets = datamaster.master 
# find datasets that have continuous exposure
cts = datasets.index[datasets['exposure'] == 'continuous'].values


### Run spatial and spatial+ for each environment, and dataset in parallel

Results are being saved in out.csv.

In [3]:
# for each environment
for envname in cts:
    env = SpaceEnv(str(envname))
    # for each masked variable
    dataset_list = list(env.make_all())
    args = zip(dataset_list, range(len(dataset_list)), [envname]*len(dataset_list))
    #print(type(args[0]))
    #print(args)
    with Pool(4) as p: # 4 is the number of processes
        pool_outputs = np.column_stack(
        (
                p.imap(erf_spatial, # does not preserve order of datasets
                            args)
                        # should write to out.csv a vector of envname, dataset id, confounding, smoothness, 
                        # erf_error_spatial, erf_error_spatialplus, pehe_spatial, pehe_spatialplus
            )
        )

/var/folders/vh/scv5k4_d0dsg0mt77xcvwv_m0000gn/T/ipykernel_47482/3571497539.py:10: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  pool_outputs = np.column_stack(


AttributeError: 'tuple' object has no attribute 'treatment'

### Analyze results by env: NOT TESTED

In [13]:
envname = cts[0]
outputs = np.loadtxt('out.csv', delimiter=',', skiprows=1)
outputs = outputs[outputs[:, 0] == envname]

smoothness_scores = outputs[:, 2]
confounding_scores = outputs[:, 3] 
erf_error_spatial = outputs[:, 4]
erf_error_spatialplus = outputs[:, 5]
pehe_spatial = outputs[:, 6]
pehe_spatialplus = outputs[:, 7]

erf_errors = dict(
    smoothness=["low" if x < 0.5 else "high" for x in smoothness_scores],
    confounding=["low" if x < 0.1 else "high" for x in confounding_scores],
    spatial_erf_error=erf_error_spatial,
    spatialplus_erf_error=erf_error_spatialplus,
    spatial_pehe_avg = pehe_spatial,
    spatialplus_pehe_avg = pehe_spatialplus
)
erf_errors = pd.DataFrame(erf_errors)
erf_errors.groupby(["smoothness", "confounding"]).agg(["mean", "std"])

spatial             spatialplus            
                              mean         std        mean         std
smoothness confounding                                                
high       low          425.494339  423.796668  425.965964  424.277590
low        high         206.488579    2.105307  207.958599    2.153853